In [94]:
class Quotient_Ring:
    def __init__(self,n,p,N,d):
        ff.<gen>=GF(p,modulus="primitive") # coeficient field
        self.gen=gen
        self.ff=ff
        assert N%n==0, "{}|{} is false".format(n,N)
        self.p=p
        self.N=N
        self.n=n
        self.d=d
    def __str__(self):
        return "Quotient Ring: ({}) / (x^{}-(w_{})^{})".format(self.ff,self.n,self.N,self.d)
    def denominator(self):
        pr=self.ff["x"]
        x=pr("x")
        return x^self.n-E(self.N)^(self.d)
    def root(self,degree):
        assert (self.p-1)%degree==0, (self.p,degree)
        return self.gen^((self.p-1)/degree)
    def split(self,k):
        return Quotient_Ring(self.n//k,self.p,self.N,self.d)

def get_prime(n):
    N=n
    while not (N+1) in Primes():
        N+=n
    return N+1

def dft(x,qr):
    n=len(x)
    w=qr.root(n)
    #print("DFT_{}: w={}".format(n,w))
    return [sum(x[i]*w^(i*j) for i in range(n)) for j in range(n)]

def split_2_2(x,qr):
    if len(x)==1:
        return x
    if len(x)==2:
        return dft(x,qr)
    n=len(x)
    w=qr.root(n)
    
    u= [x[2*i] for i in range(n//2)]
    U = split_2_2(u,qr.split(2))
    
    r = [[x[2*(2*i+l)+1] for i in range(n//4)] for l in range(2)]
    R = [split_2_2(r[l],qr.split(4)) for l in range(2)]
    
    s = [[w^((2*l+1)*i)*R[l][i] for i in range(n//4)] for l in range(2)]
    S = [split_2_2([s[l][i] for l in range(2)],qr.split(n//2)) for i in range(n//4)]
    
#     print("u=x[4i]")
#     print(u)
#     print("U=DFT(u)")
#     print(U)
#     print("r=x[4i+2l+1]")
#     print(r)
#     print("R_l=DFT(r_l)")
#     print(R)
#     print("s=w^((2l+1)*i)*DFT(r_l)_i")
#     print(s)
#     print("S=DFT(s)")
#     print(S)
    X = [0 for i in range(n)]
    for i in range(n//4):
        for j in range(4):
            t0=U[(i+(j%2)*n//4)]
            t1=qr.root(4)^j*S[i][j%2]
            #print(i,j,t1)
            X[i+j*n//4]=t0+t1
    return X


In [93]:
math.log(16,2)%1==0

True

In [95]:
n=16
p=17
qr=Quotient_Ring(n,p,n,n)
x=[i for i in range(n)]
print(dft(x,qr))
print(split_2_2(x,qr))

DFT_16: w=3
[1, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
DFT_2: w=16
[1, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]


In [87]:
def split_k_c(x,qr,k,c):
    n=len(x)
    assert n%k==0 and k%c==0, f"! {c}|{k}|{n}"
    w=qr.root(n)
    
    u = [x[c*i] for i in range(n//c)]
    U = dft(u,qr)
    
    # r -> t,l,i
    r = [[[x[k*i+c*l+t] for i in range(n//k)] for l in range(k//c)] for t in range(1,c)]
    # R -> t,l,i'
    R = [[dft(x[t][l]) for l in range(k//c)] for t in range(1,c)]
    
    # s -> t,l,j,g
    s = [[[w^(c*l*g)*R[t][l][g] for g in range(n//k)] for l in range(k//c)] for t in range(c)]
    # s_l -> t,j,g,l
    s_l = [[[s[t][l][j][g] for l in range(k//c)] for g in range(n//k)] for t in range(1,c)]
    # S -> t,j,g,l'
    S = [[dft(s_l[t][j][g]) for g in range(n//k)] for t in range(1,c)]

    print("u=x[ci]")
    print(u)
    print("U[j]=DFT(u)_j")
    print(U)
    print("r(t,l,i)=x[ki+cl+t]")
    print(r)
    print("R(t,l,j)=DFT(r(t,l,_))_j")
    print(R)
    print("s(t,l,g)=w^(clg)*R(t,l,g)")
    print(s)
    print("s_l(t,g,l)=s(t,l,g)")
    print(s_l)
    print("S(t,g,l)=DFT(s_l(t,g,_)_l)")
    print(S)
    
    X = [0 for i in range(n)]
    for i in range(n//(2*k)):
        for j in range(2*k):
            t0=U[(i+j*n//(2*k))%(n//k)]
            t1=qr.root(2*k)^j*S[i][j%k]
            print((i,j),(S[i][j%k],t1))
            X[i+j*n//(2*k)]=t0+t1
    return X
    
def split_k_2(x,qr,k):
    n=len(x)
    w=qr.root(n)
    
    u= [x[k*i] for i in range(n//k)]
    U = dft(u,qr.split(k))
    
    r = [[x[2*(k*i+l)+1] for i in range(n//(2*k))] for l in range(k)]
    R = [dft(r[l],qr.split(2*k)) for l in range(k)]
    
    s = [[w^((2*l+1)*i)*R[l][i] for i in range(n//(2*k))] for l in range(k)]
    S = [dft([s[l][i] for l in range(k)],qr.split(n//k)) for i in range(n//(2*k))]
    
    print("u=x[ki]")
    print(u)
    print("U=DFT(u)")
    print(U)
    print("r=x[2(ki+l)+1]")
    print(r)
    print("R_l=DFT(r_l)")
    print(R)
    print("s=w^((2l+1)*i)*DFT(r_l)_i")
    print(s)
    print("S=DFT(s)")
    print(S)
    X = [0 for i in range(n)]
    for i in range(n//(2*k)):
        for j in range(2*k):
            t0=U[(i+j*n//(2*k))%(n//k)]
            t1=qr.root(2*k)^j*S[i][j%k]
            print((i,j),(S[i][j%k],t1))
            X[i+j*n//(2*k)]=t0+t1
    return X

In [89]:
n=6
p=13
k=3
qr=Quotient_Ring(n,p,n,n)
x=[i for i in range(n)]
print(dft(x,qr))
print(split_k_2(x,qr,k))

DFT_6: w=4
[2, 2, 3, 10, 4, 5]
DFT_2: w=12
DFT_1: w=1
DFT_1: w=1
DFT_1: w=1
DFT_3: w=3
u=x[ki]
[0, 3]
U=DFT(u)
[3, 10]
r=x[2(ki+l)+1]
[[1], [3], [5]]
R_l=DFT(r_l)
[[1], [3], [5]]
s=w^((2l+1)*i)*DFT(r_l)_i
[[1], [3], [5]]
S=DFT(s)
[[9, 3, 4]]
(0, 0) (9, 9)
(0, 1) (3, 12)
(0, 2) (4, 12)
(0, 3) (9, 4)
(0, 4) (3, 1)
(0, 5) (4, 1)
[12, 9, 2, 1, 4, 11]


In [83]:
def split(x,base):
    if len(x)==base:
        return x
    y=[[x[2*i] for i in range(len(x)//2)],split([x[2*i+1] for i in range(len(x)//2)],base)]
    return y

def rec_ms(x,qr):
    #print(x)
    if len(x[0])==1:
        return x,0
    X_0,mults_0 = meta_split(x[0],qr)
    X_1,mults_1 = rec_ms(x[1],qr)
    return [X_0,X_1],mults_0+mults_1


def combine(y,qr,depth):
    #print("comb: incoming",y)
    if len(y[0])==1:
        S = [y[0][0]+y[1][0],y[0][0]-y[1][0]]
        #print(depth,S)
        return S,0
    S_dp1,mults = combine(y[1],qr,depth+1)
    n=len(y[0])
    s = [qr.root(2*n)^i*S_dp1[i] for i in range(n)]
    #S = [y[0][i%n]+qr.root(2*n)^i*S_dp1[i%n] for i in range(2*n)]
    S = [y[0][i]+s[i] for i in range(n)]+[y[0][i]-s[i] for i in range(n)]
    #print("partial",[qr.root(2*n)^i*S_dp1[i%n] for i in range(2*n)])
    #print(depth,S)
    return S,mults+n-1

def meta_split(x,qr):
    n=len(x)
    if n==1:
        return x,0
    assert math.log(n,2)%1==0
    y=split(x,1)
    Y,mults0=rec_ms(y,qr)
    #print(Y)
    #print("combining")
    X,mults1=combine(Y,qr,0)
    print(len(X),mults0+mults1)
    return X,mults0+mults1
    
    
n=8
p=get_prime(n)
qr=Quotient_Ring(n,p,n,n)
x=[i*3 for i in range(n)]
#print(split(x,1))
print(meta_split(x,qr))
print(dft(x,qr))

2 0
4 1
2 0
8 5
([16, 3, 2, 9, 5, 1, 8, 7], 5)
DFT_8: w=9
[16, 3, 2, 9, 5, 1, 8, 7]


In [84]:
Z=10
res = [0 for i in range(Z)]
for i in range(Z):
    n=2^i
    p=get_prime(n)
    qr=Quotient_Ring(n,p,n,n)
    x=[i for i in range(n)]
    #print(split(x,1))
    X,mult = meta_split(x,qr)
    res[i]=mult
print(res)

2 0
2 0
4 1
2 0
4 1
2 0
8 5
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
32 49
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
32 49
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
64 129
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
32 49
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
64 129
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
32 49
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
128 321
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
32 49
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
64 129
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
32 49
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
128 321
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
32 49
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0
8 5
2 0
4 1
2 0
64 129
2 0
4 1
2 0
8 5
2 0
4 1
2 0
16 17
2 0
4 1
2 0


In [82]:
[2^i*i for i in range(Z)]

[0, 2, 8, 24, 64, 160, 384, 896, 2048, 4608]

In [58]:

def get_prime(n):
    N=n
    while not (N+1) in Primes():
        N+=n
    return N+1
get_prime(32)

97